In [226]:
from bs4 import BeautifulSoup
import requests
import re
import urllib
import lxml.html


In [248]:
base_url_pol = 'http://www.mof.gov.cn/zhengwuxinxi/zhengcefabu/index.htm'

In [228]:
def get_links_from_main_url(base_url):
    """
    Take a base_url and return a list of links on the page
    
    """
    #return a connection to some base_url
    connection = urllib.request.urlopen(base_url)
    #take the html object and get its dom structure
    dom =  lxml.html.fromstring(connection.read())
    #we need something to hold our urls
    urls = []
    #iterate over dom and get the href content
    for path in dom.xpath('//a/@href'):
        #if the item in the that object matches the pattern for a url append to our list
        if path.startswith('http://') and path.endswith('.html'):
            urls.append(path)
    return urls

In [256]:
#do some basic data cleaning
def blob_gen(ps):
    """
    Takes a Beautiful Soup result set object of 'p' elements and removes unicode object encoding
    
    Parameters
    ----------
    ps: BSoup Result Set list
    
    Returns
    -------
    string
    """
    blob = ''
    for p in ps[1:]:
        blob += str(p.text.replace('\u3000', u' ').replace('\xa0', u' '))
    return blob

In [257]:
def get_metadata(policy_url):
    """
    Given a url returns a dictionary of key, value pairs of relevant metadata
    
    >>>get_metadata('http://www.some_url_from_some_website.html')
    >>>{
    ...'URL': 'http://www.some_url_from_some_website.html', 
    ...'title': 'Epic Surf on the North Shore', 
    ...'policy_id': 'Department of Stoke #234,
    ...'date_published': 20161129, 
    ...'content': 'Stay stoked and be happy!'
        }
    
    """
    #get html object for soup
    html = requests.get(str(policy_url))
    #intermediate object to Soupify and store the content from the page
    soup = BeautifulSoup(html.content, 'html.parser')
    #Parse the url and pull out the date of publish
    date_published = re.search(r'([0-9]{8})', str(policy_url)).group()
    #find the title 
    title = soup.find('td', class_='font_biao1').text.strip()
    #get all the p elements where the text lives
    ps = soup.find('div', class_='TRS_Editor').find_all('p')
    #take the zeroth element where the policy id lives
    policy_id = ps[0].text
    
    #get content
    blob = blob_gen(ps)
    
    #construct our entry
    entry = {
    'URL': str(policy_url), 
     'title': title, 
     'policy_id': policy_id,
     'date_published': date_published, 
    'content': blob
          
    }
    
    return entry

In [174]:
rem = re.search(r'([0-9]{8})', str(pol_urls[0]))
date_published = rem.group()

In [177]:
title = soup.find('td', class_='font_biao1').text.strip()
ps = soup.find('div', class_='TRS_Editor').find_all('p')
policy_nem = ps[0].text

In [267]:
#Get list of urls
type(ps[1].text)

str

In [285]:
def gen_metadata_iter(base_url_pol):
    for i, _ in enumerate(get_links_from_main_url(base_url_pol)):
        print('Writing entry {} from url: {}'.format(i,_))
        yield get_metadata(_)

In [297]:
for i in range(1, 23):
    base_url = base_url_pol.replace('.htm', '_' + str(i) + '.htm')
    for _ in gen_metadata_iter(base_url):
        print(_)


Writing entry 0 from url: http://jrs.mof.gov.cn/zhengwuxinxi/zhengcefabu/201610/t20161013_2435127.html
{'URL': 'http://jrs.mof.gov.cn/zhengwuxinxi/zhengcefabu/201610/t20161013_2435127.html', 'date_published': '20161013', 'policy_id': '财金[2016]91号', 'content': '各省、自治区、直辖市、计划单列市财政厅（局）、教育厅（局）、科学技术厅（局）、工业和信息化厅（局、委）、民政厅（局）、人力资源社会保障厅（局）、国土资源厅（局）、环境保护厅（局）、交通运输厅（局、委）、水利厅（局）、农业厅（局）、商务厅（局）、文化厅（局）、卫生和计划生育委员会、体育局、林业局、旅游局、能源局、铁路监督管理局、民用航空局，新疆生产建设兵团财务局、教育局、科技局、工业和信息化委员会、民政局、人力资源社会保障局、国土资源局、交通局、水利局、农业局、商务局、卫生局：  为进一步推进政府和社会资本合作（以下简称PPP）工作取得实质性进展，发挥示范项目引领作用，调动社会资本参与积极性，财政部会同相关部门联合启动了第三批政府和社会资本合作示范项目申报筛选工作。现将评审结果及有关事宜通知如下：  一、经有关省、自治区、直辖市、计划单列市和部委推荐及专家评审，现确定北京市首都地区环线高速公路（通州―大兴段）等516个项目作为第三批PPP示范项目（详见附件），计划总投资金额11,708亿元。  二、示范项目所在省、自治区、直辖市、计划单列市财政部门和相关行业主管部门要高度重视，密切协作配合。按照国务院和财政部等部门出台的相关制度文件要求，依法择优选择社会资本，鼓励同等条件下优先选择民营资本；规范推进项目实施，落实示范项目责任制，建立对口联系和跟踪管理机制，确保示范项目实施质量。  三、各级财政部门要切实履行财政管理职能，积极为项目加快落地创造条件。要做好物有所值评价和财政承受能力论证工作，优化工作流程，提高效率；因地制宜给予前期费用补助、以奖代补等资金支持，协调推动示范项目与中国政企合作投资基金进行合作对接；会同有关部门统筹安排财政资金、国有资产等各类公共

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [283]:
with open('pol.txt', 'w+') as f:
    for url in urls:
        entry = get_metadata(url)
    f.writelines(entry)
        

In [284]:
import json
import io
with open('some_new_file.txt', 'w+') as f:
    for url in urls:
        json.dumps(get_metadata(url),ensure_ascii=False)

In [276]:
json.dumps(get_metadata(get_links_from_main_url(base_url_pol)[0]), ensure_ascii=False)

'{"URL": "http://gks.mof.gov.cn/zhengfuxinxi/guizhangzhidu/201611/t20161128_2467572.html", "date_published": "20161128", "policy_id": "财办库[2016]416号", "content": "党中央有关部门办公厅（室），国务院各部委、各直属机构办公厅（室），全国人大常委会办公厅秘书局，全国政协办公厅秘书局，高法院办公厅，高检院办公厅，各民主党派中央办公厅（室），有关人民团体办公厅（室）：  为简化优化中央预算单位变更政府采购方式和采购进口产品审批审核程序，提高审批审核工作效率，保障中央预算单位政府采购活动的顺利开展，现将有关事宜通知如下：  一、推行变更政府采购方式一揽子申报和批复  主管预算单位应加强本部门变更政府采购方式申报管理，定期归集所属预算单位申请项目，向财政部（国库司）一揽子申报，财政部（国库司）一揽子批复。归集的周期和频次由主管预算单位结合实际自行确定。时间紧急或临时增加的采购项目可单独申报和批复。  二、推行采购进口产品集中论证和统一报批  主管预算单位应按年度汇总所属预算单位的采购进口产品申请，组织专家集中论证后向财政部（国库司）申报，财政部（国库司）统一批复。时间紧急或临时增加的采购项目可单独申报和批复。  三、提高申报和审批审核效率  主管预算单位应完善内部管理规定和流程，明确时间节点和工作要求，及时做好所属预算单位变更政府采购方式和采购进口产品申报工作。对于中央预算单位变更政府采购方式和采购进口产品申请，财政部（国库司）实行限时办结制。对于申请理由不符合规定的项目，财政部（国库司）及时退回并告知原因；对于申请材料不完善和不符合规定的，财政部（国库司）一次性告知主管预算单位修改补充事项；对于符合规定的申请项目，财政部（国库司）自收到申请材料起５个工作日内完成批复。  中央预算单位变更政府采购方式和采购进口产品的其他事宜，按照《财政部关于印发＜中央预算单位变更政府采购方式审批管理办法＞的通知》（财库〔２０１５〕３６号）、《财政部关于印发＜政府采购进口产品管理办法＞的通知》（财库〔２００７〕１１９号）和《财政部关于完善中央单位政府采购预算管理和中央高校、科研院所科研仪器设备采购管理有关事项的通知》（财库〔２０１６〕１